<a href="https://colab.research.google.com/github/vishwasjoshi2019/Stock_Market_Analysis/blob/main/Volatility_Model_S%26P(GARCH).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install arch

In [ ]:
import yfinance as yf
import pandas as pd
import arch
import matplotlib.pyplot as plt

import plotly.graph_objs as go

import numpy as np

from sklearn.metrics import r2_score



In [ ]:

# Define the ticker symbol for the S&P 500
symbol = "^GSPC"

# Define the date range for the past 2 years
end_date = pd.to_datetime('2023-09-29')  # Replace with the desired end date
start_date = end_date - pd.DateOffset(years=2)  # 2 years of historical data

# Fetch historical data
sp500_data = yf.download(symbol, start=start_date, end=end_date)

# Calculate daily returns
sp500_data['Returns'] = sp500_data['Adj Close'].pct_change().dropna()
missing_values = sp500_data['Returns'].isna().sum()
print(f"Number of missing values in Returns column: {missing_values}")
# Linear interpolation to fill missing values
sp500_data['Returns'].interpolate(method='linear', inplace=True)

sp500_data[sp500_data['Returns'].isna()]

[*********************100%%**********************]  1 of 1 completed
Number of missing values in Returns column: 1


,Open,High,Low,Close,Adj Close,Volume,Returns
Date,,,,,,,
2021-09-29,4362.410156,4385.569824,4355.080078,4359.459961,4359.459961,3712660000,NaN


In [ ]:
# Identify rows with NaN in the 'Returns' column
rows_with_nan = sp500_data[sp500_data['Returns'].isna()]

# Remove rows with NaN in the 'Returns' column
sp500_data.drop(rows_with_nan.index, inplace=True)

In [ ]:
# Rescale the 'Returns' data by multiplying it by 100
sp500_data['Returns'] = sp500_data['Returns'] * 100

# Create a GARCH(1,1) model
model = arch.arch_model(sp500_data['Returns'], vol='Garch', p=2, q=2)

# Fit the model
results = model.fit()

# Print model summary
print(results.summary())

# Get the conditional volatility forecast
forecast_horizon = len(sp500_data)  # Forecast the entire dataset
forecasted_volatility = results.forecast(start=0, horizon=forecast_horizon)

# # Plot historical and forecasted volatility
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# plt.plot(sp500_data.index, sp500_data['Returns'], label='Daily Returns', alpha=0.7)
# plt.plot(sp500_data.index, forecasted_volatility.variance.values[-1, :], label='Forecasted Volatility', alpha=0.7)
# plt.xlabel('Date')
# plt.ylabel('Value')
# plt.title('S&P 500 Volatility Model')
# plt.legend()
# plt.show()


Iteration:      1,   Func. Count:      8,   Neg. LLF: 2686.2141624617725
Iteration:      2,   Func. Count:     18,   Neg. LLF: 81454.35295329984
Iteration:      3,   Func. Count:     28,   Neg. LLF: 781.9274689779421
Iteration:      4,   Func. Count:     36,   Neg. LLF: 780.675872833619
Iteration:      5,   Func. Count:     44,   Neg. LLF: 779.2900420939463
Iteration:      6,   Func. Count:     52,   Neg. LLF: 779.9764618753861
Iteration:      7,   Func. Count:     60,   Neg. LLF: 778.7996505070248
Iteration:      8,   Func. Count:     68,   Neg. LLF: 778.6183493210619
Iteration:      9,   Func. Count:     75,   Neg. LLF: 778.6246668241524
Iteration:     10,   Func. Count:     83,   Neg. LLF: 779.3152321838451
Iteration:     11,   Func. Count:     92,   Neg. LLF: 778.606783102811
Iteration:     12,   Func. Count:    100,   Neg. LLF: 778.5934631939577
Iteration:     13,   Func. Count:    108,   Neg. LLF: 778.5910240668842
Iteration:     14,   Func. Count:    115,   Neg. LLF: 778.5910208

In [ ]:
import plotly.graph_objs as go

# Create interactive plot using Plotly
fig = go.Figure()

# Plot previous returns
fig.add_trace(go.Scatter(x=sp500_data.index, y=sp500_data['Returns'], mode='lines', name='Daily Returns'))

# Plot historical volatility
fig.add_trace(go.Scatter(x=sp500_data.index, y=sp500_data['Returns'].rolling(window=20).std(), mode='lines', name='Historical Volatility'))

# Plot future predicted volatility
fig.add_trace(go.Scatter(x=sp500_data.index, y=forecasted_volatility.variance.values[-1, :], mode='lines', name='Predicted Volatility'))

# Customize the layout
fig.update_layout(
    title='S&P 500 Volatility and Returns',
    xaxis_title='Date',
    yaxis_title='Value',
    showlegend=True
)

# Show the interactive plot
fig.show()

In [ ]:
# Calculate the squared errors
squared_errors = (forecasted_volatility.variance.values[-1, :] - sp500_data['Returns'].rolling(window=20).std())**2

# Calculate Mean Squared Error (MSE)
mse = np.mean(squared_errors)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# Calculate Mean Absolute Error (MAE)
mae = np.mean(np.abs(forecasted_volatility.variance.values[-1, :] - sp500_data['Returns'].rolling(window=20).std()))

# Print the accuracy metrics
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

Mean Squared Error (MSE): 0.2855
Root Mean Squared Error (RMSE): 0.5343
Mean Absolute Error (MAE): 0.4460


In [ ]:
import plotly.graph_objs as go

# Create a time index for the forecasted volatility
forecast_index = pd.date_range(start=sp500_data.index[-1], periods=forecast_horizon, freq='B')

# Create a Plotly figure
fig = go.Figure()

# Plot historical returns (optional)
fig.add_trace(go.Scatter(x=sp500_data.index, y=sp500_data['Returns'], mode='lines', name='Daily Returns'))

# Plot predicted volatility for the next 3 months
fig.add_trace(go.Scatter(x=forecast_index, y=np.sqrt(forecasted_volatility.variance.values[-1, :]) * 100, mode='lines', name='Predicted Volatility'))

# Customize the layout
fig.update_layout(
    title='S&P 500 Predicted Volatility for Next 3 Months',
    xaxis_title='Date',
    yaxis_title='Volatility (%)',
    showlegend=True
)

# Show the interactive plot
fig.show()
